In [85]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
from bosonnlp import BosonNLP
import pickle
import numpy as np
import itertools
from scipy.sparse import csr_matrix
from collections import OrderedDict


%matplotlib inline

In [4]:
nlp = BosonNLP('kNKCfN_V.4919.-3H8F-XH-BQk')

In [23]:
def clean_text(s):
    s = s.replace(" ","")
    s = s.replace("\n","")
    return s

news_day = os.listdir("C:/Users/thtang/Desktop/NLP_for_Fintech/SysJustNews/news-vip/")
news_id = []
news_complete_paths = []
for day in news_day:
    path = os.path.join("C:/Users/thtang/Desktop/NLP_for_Fintech/SysJustNews/news-vip/",day)
    text_id_list = os.listdir(path)
    news_id+= text_id_list
    for i in text_id_list:
        news_complete_paths.append(os.path.join(path,i))
news_complete_paths = np.array(news_complete_paths)

In [38]:
select_path_index = np.random.randint(0,len(news_complete_paths),493)

In [39]:
select_news = news_complete_paths[select_path_index]

In [40]:
doc_NER = []
for path in select_news:
    with open(path,"r", encoding = 'utf8') as f:
        text = clean_text(f.read())
    soup = BeautifulSoup(text)
    ner_output = nlp.ner(soup.text, sensitivity=2)
    doc_NER.append(ner_output)

In [45]:
pickle.dump(doc_NER, open("./doc_NER.pkl","wb"))

In [73]:
ner_corpus = []
for i in range(len(doc_NER)):
    ner_doc = []
    for tup in doc_NER[i][0]["entity"]:
        if tup[2]=="company_name" or tup[2]=="product_name":
            word = doc_NER[i][0]["word"][tup[0]:tup[1]]
            ner_doc+=word
    if ner_doc != []:
        ner_corpus.append(ner_doc)

In [86]:
names = list(set([w for l in ner_corpus for w in l]))
occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)
 
# Find the co-occurrences:
for l in ner_corpus:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1
            
co_occur = pd.DataFrame.from_dict(occurrences)

In [89]:
co_occur.to_csv('./fin_cooccurancy_matrix.csv')